In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup


from urllib.request import urlopen
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

from urllib.parse import quote 
import re

hira=['い','う','え','お','は','ひ','ふ','へ','ほ',
'か','き','く','け','こ','ま','み','む','め','も',
'さ','し','す','せ','そ','や','ゆ','よ',
'た','ち','つ','て','と','ら','り','る','れ','ろ',
'な','に','ぬ','ね','の','わ','を','ん']
pre_url="https://ja.wikipedia.org/w/index.php?title="
pp_pre_url="https://ja.wikipedia.org"

def get_html_from_url(fromurl):
    html = urlopen(fromurl)
    bs_object = BeautifulSoup(html, "html.parser")
    return bs_object

# get name list and url

In [ ]:
#get name list
category_query="%E4%B8%89%E5%9B%BD%E5%BF%97%E3%81%AE%E7%99%BB%E5%A0%B4%E4%BA%BA%E7%89%A9"

href_df=pd.DataFrame(columns=["href", "title"])
#from each hiragana name index
for i in hira:
    bsObject=get_html_from_url( pre_url+"Category:"+category_query+"&from="+quote(i) ) 
    tags = bsObject.find_all('div', attrs={'class': 'mw-category-group'})
                     
    for j in range(0,len(tags)):
        ahreflist=tags[j].findAll(href=True)
        for k in range(0,len(ahreflist)):
            href_df=href_df.append(pd.DataFrame(columns=["href", "title"],
                data=[[ahreflist[k].attrs["href"],ahreflist[k].attrs["title"]]]))
#drop duplication
href_df=href_df.drop_duplicates()


# get each page

In [ ]:
#get people page (not category)
peeps_url=href_df[~href_df["title"].str.contains("Category")]

peeps_url["href"]=pp_pre_url+peeps_url["href"]
peeps_url=peeps_url.reset_index(drop=True)

#add columns and initialize 
peeps_url["content"]=peeps_url["category"]=peeps_url["menu"]=peeps_url["table"]=peeps_url["table_lst"]=''

ind=0
for i in peeps_url["href"]: 
    bsObject2=get_html_from_url( i ) 
    
    whole=bsObject2.find_all('div',attrs={'class': 'mw-parser-output'})[0].text
    
    #append the category elements like sub-list
    categories=bsObject2.find_all('div',attrs={'class': 'mw-normal-catlinks'})[0].find_all("a")
    category=[]
    for j in range(0,len(categories)):
        category.append(categories[j].text)
    
    #contents index of wiki page
    if len(bsObject2.find_all('div', attrs={'class': 'toc'}) )>0:
        menu=bsObject2.find_all('div', attrs={'class': 'toc'})[0].text
    else:
        menu=''
    
    #characteristic table under the page. wiki page's category(hyperlinks)
    if len(bsObject2.find_all('table', attrs={'class': 'infobox'}) )>0:
        tables=bsObject2.find_all('table', attrs={'class': 'infobox'})[0].text
        table_contents=bsObject2.find_all('table',
                                          attrs={'class': 'infobox'})[0].find_all("tr")
    else:
        tables=table_contents=''
    table_list=[]
    for j in range(0,len(table_contents)): 
        table_txt=table_contents[j].text
        if table_txt.startswith("\n"):
            continue
        elif "\n" in table_txt:
            table_list.append(table_txt.split("\n"))
            
    peeps_url.loc[ind,"content"]= whole
    peeps_url.loc[ind,"category"]= category
    peeps_url.loc[ind,"menu"]= menu
    peeps_url.loc[ind,"table"]= tables
    peeps_url.loc[ind,"table_lst"]= table_list
    
    ind=ind+1


# preprocessing

In [ ]:
# in name(title), remove after (
peeps_url["title"]=peeps_url["title"].str.replace("\(.*", "") 

peeps_url=peeps_url[peeps_url["content"].str.len()>0]

#remove unnecessary phrases
peeps_url["content"]=peeps_url["content"].apply(lambda x: re.sub(r'参考文献\[編集\][\s\S]*',"",x))
peeps_url["content"]=peeps_url["content"].apply(lambda x: re.sub(r'出典\[編集\][\s\S]*',"",x))
peeps_url["content"]=peeps_url["content"].apply(lambda x: re.sub(r'この項目は、中国の歴史に関連した書き[\s\S]*',"",x))
peeps_url["content"]=peeps_url["content"].apply(lambda x: re.sub(r'参考資料\[編集\][\s\S]*',"",x))


In [ ]:
#ready to create pivot. weightdf's index by title column
weightdf=peeps_url[["title"]]

#create pivot by in-contents frequency
def get_weight(nm):
    global weightdf
    weightdf[nm]=peeps_url["content"].str.count(nm) 
peeps_url["title"].apply(lambda x: get_weight(x))

In [ ]:
#pivot to long 
weightdf.index=weightdf["title"]
weightdf=weightdf.drop("title",axis=1)
weightdf=weightdf.stack().reset_index()
#long df's column name name1(source) name2(target) weight of edge
weightdf.columns=["name1", "name2", "weight"]

#prep
weightdf=weightdf[weightdf["name1"]!=weightdf["name2"]]
weightdf=weightdf[weightdf["weight"]>0].reset_index(drop=True)

weightdf=weightdf.groupby(["name1","name2"]).sum().reset_index()

In [ ]:
#create node size(cnt) df
node_cnt_df=weightdf["name1"].append(weightdf["name2"]).value_counts().reset_index()
node_cnt_df.columns=["node_name", "cnt"]

# network 

In [ ]:
import networkx as nx
import nxviz as nxv


# create the networkx object to get coordinate position
G = nx.from_pandas_edgelist(weightdf,  
                            source='name1', target='name2' ,edge_attr="weight") 

pos = nx.spring_layout(G)

In [ ]:
def node_and_edge(interaction_df, node_coord, node_cnt_df):
    node_edge_df1=interaction_df.merge(node_coord.reset_index(), left_on="name1", right_on="index", how="inner")
    node_edge_df2=interaction_df.merge(node_coord.reset_index(), left_on="name2", right_on="index", how="inner")

    node_edge_merge=node_edge_df1.append(node_edge_df2) 
    
    node_edge_merge2=node_edge_merge.merge(node_cnt_df, left_on="name1", right_on="node_name", how="inner")
    
    node_edge_merge2=node_edge_merge2.drop("index", axis=1)
    node_edge_merge2=node_edge_merge2.drop("node_name", axis=1)
    node_edge_merge2=node_edge_merge2.rename(columns={0:"coord_x", 1:"coord_y", "cnt":"name1_cnt"})
    
    #edge name key. source_target 
    node_edge_df["edge_name"]=node_edge_df["name1"]+"_"+node_edge_df["name2"]
    node_ege_df=node_ege_df.drop_duplicates("edge_name")
    
    return node_ege_df

In [ ]:
node_edge_df=node_and_edge(weightdf, pd.DataFrame(pos).transpose(), node_cnt_df)

In [ ]:
node_ege_df.to_csv("node_and_edge.csv", encoding="utf-8", index=False) 

### visualization in
### https://public.tableau.com/profile/yj.choi#!/vizhome/easternHaneranetworkplot/1_1